In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
# prevent ultralytics from tracking your activity
!yolo settings sync=False
import ultralytics
ultralytics.checks()

In [ ]:
!yolo task=detect mode=predict model=yolo11m-seg.pt conf=0.25 source='https://media.roboflow.com/notebooks/examples/dog.jpeg' save=True

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/segment/predict/dog.jpg', width=600)

In [ ]:
from ultralytics import YOLO
from PIL import Image
import requests

model = YOLO('yolo11m-seg.pt')
image = Image.open(requests.get('https://media.roboflow.com/notebooks/examples/dog.jpeg', stream=True).raw)
result = model.predict(image, conf=0.25)[0]

In [ ]:
from roboflow import Roboflow

!roboflow workspace list

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow

ROBOFLOW_API_KEY = userdata.get('ROBOFLOW_API_KEY')
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

workspace = rf.workspace("tbe") # Your workspace ID
project = workspace.project("rice-grain-svjri") # Your project ID
version = project.version(4)
dataset = version.download("yolov11")

In [ ]:
%cd {HOME}

!yolo task=segment mode=train model=yolo11m-seg.pt data={dataset.location}/data.yaml epochs=1000 imgsz=640 plots=True

In [ ]:
!ls {HOME}/runs/segment/train/

In [ ]:
# from IPython.display import Image as IPyImage

# IPyImage(filename=f'{HOME}/runs/segment/train/confusion_matrix.png', width=600)

In [ ]:
!yolo task=segment mode=val model={HOME}/runs/segment/train/weights/best.pt data={dataset.location}/data.yaml imgsz=640 plots=True max_det=1000

In [ ]:
# Data Augmentation

# Melakukan augmentasi pada dataset untuk meningkatkan variasi data training

In [ ]:
# Install library untuk augmentasi
%pip install albumentations opencv-python-headless

In [ ]:
import cv2
import numpy as np
import albumentations as A
from pathlib import Path
import shutil
from tqdm import tqdm
import yaml
from collections import Counter

# Fungsi untuk membaca annotation YOLO format
def read_yolo_annotation(annotation_path):
    with open(annotation_path, 'r') as f:
        annotations = []
        for line in f.readlines():
            parts = line.strip().split()
            if len(parts) > 0:
                class_id = int(parts[0])
                coords = [float(x) for x in parts[1:]]
                annotations.append([class_id] + coords)
    return annotations

# Fungsi untuk menulis annotation YOLO format
def write_yolo_annotation(annotation_path, annotations):
    with open(annotation_path, 'w') as f:
        for ann in annotations:
            class_id = int(ann[0])
            coords = ' '.join([f'{x:.6f}' for x in ann[1:]])
            f.write(f'{class_id} {coords}\n')

# Fungsi untuk transformasi koordinat polygon
def transform_polygon_coords(coords, img_width, img_height, transform_matrix):
    """Transform polygon coordinates using transformation matrix"""
    transformed_coords = []
    for i in range(0, len(coords), 2):
        x = coords[i] * img_width
        y = coords[i + 1] * img_height

        # Apply transformation
        if transform_matrix == 'horizontal_flip':
            x = img_width - x
        elif transform_matrix == 'vertical_flip':
            y = img_height - y
        elif transform_matrix == 'rotate_90':
            x, y = img_height - y, x
        elif transform_matrix == 'rotate_180':
            x, y = img_width - x, img_height - y
        elif transform_matrix == 'rotate_270':
            x, y = y, img_width - x

        # Normalize back
        x_norm = x / img_width if transform_matrix not in ['rotate_90', 'rotate_270'] else x / img_height
        y_norm = y / img_height if transform_matrix not in ['rotate_90', 'rotate_270'] else y / img_width

        transformed_coords.extend([x_norm, y_norm])

    return transformed_coords

# Fungsi untuk apply transformasi individual dengan tracking
def apply_tracked_transform(image, annotations, img_width, img_height):
    """Apply transformation dan track transformasi untuk koordinat"""
    applied_transforms = []

    # Apply HorizontalFlip
    if np.random.random() < 0.5:
        image = cv2.flip(image, 1)
        applied_transforms.append('horizontal_flip')

    # Apply VerticalFlip
    if np.random.random() < 0.5:
        image = cv2.flip(image, 0)
        applied_transforms.append('vertical_flip')

    # Apply RandomRotate90
    rotate_choice = np.random.random()
    if rotate_choice < 0.5:
        k = np.random.choice([1, 2, 3])  # 90, 180, or 270 degrees
        image = np.rot90(image, k)
        if k == 1:
            applied_transforms.append('rotate_90')
        elif k == 2:
            applied_transforms.append('rotate_180')
        elif k == 3:
            applied_transforms.append('rotate_270')

    transformed_annotations = []
    for ann in annotations:
        class_id = ann[0]
        coords = ann[1:]

        current_coords = coords
        current_width = img_width
        current_height = img_height

        for transform_type in applied_transforms:
            current_coords = transform_polygon_coords(
                current_coords, current_width, current_height, transform_type
            )
            if transform_type in ['rotate_90', 'rotate_270']:
                current_width, current_height = current_height, current_width

        transformed_annotations.append([class_id] + current_coords)

    return image, transformed_annotations

# Fungsi untuk membaca konfigurasi dataset dari YAML
def load_data_config(data_yaml_path):
    data_yaml_path = Path(data_yaml_path)
    with open(data_yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    names = data.get('names', [])
    if isinstance(names, dict):
        names = [names[str(i)] for i in range(len(names))]

    base_dir = data_yaml_path.parent

    def resolve_path(path_value):
        path_value = Path(path_value)
        if not path_value.is_absolute():
            return (base_dir / path_value).resolve()
        return path_value.resolve()

    splits = {}
    for split_key in ('train', 'val', 'test'):
        split_path = data.get(split_key)
        if not split_path:
            continue
        images_dir = resolve_path(split_path)
        labels_dir = (images_dir.parent / 'labels').resolve()
        splits[split_key] = {
            'images': images_dir,
            'labels': labels_dir,
        }

    return {
        'names': names,
        'splits': splits,
    }

# Menghitung distribusi kelas dan statistik per gambar
def collect_class_stats(labels_dir, num_classes):
    labels_dir = Path(labels_dir)
    counts = Counter({cls: 0 for cls in range(num_classes)})
    image_stats = {}

    for label_path in labels_dir.glob('*.txt'):
        annotations = read_yolo_annotation(label_path)
        per_image = Counter({cls: 0 for cls in range(num_classes)})
        for ann in annotations:
            class_id = int(ann[0])
            if class_id >= num_classes:
                continue
            per_image[class_id] += 1
            counts[class_id] += 1
        image_stats[label_path.stem] = per_image

    return counts, image_stats

# Menampilkan distribusi kelas yang mudah dibaca
def print_class_distribution(split_name, counts, class_names):
    total = sum(counts.values())
    print(f"\nDistribusi kelas untuk {split_name}:")
    for idx, class_name in enumerate(class_names):
        value = counts.get(idx, 0)
        if total > 0:
            pct = (value / total) * 100
            print(f"  - {class_name}: {value} ({pct:.2f}%)")
        else:
            print(f"  - {class_name}: {value}")

# Menentukan rencana augmentasi agar kelas minoritas mendekati jumlah kelas mayoritas
def build_balanced_augmentation_plan(image_stats, target_class_id, deficit, max_aug_per_image=5):
    plan = {}
    if deficit <= 0:
        return plan

    eligible = []
    for stem, stats in image_stats.items():
        instances = stats.get(target_class_id, 0)
        if instances > 0:
            eligible.append((stem, instances))

    if not eligible:
        return plan

    eligible.sort(key=lambda item: item[1], reverse=True)
    total_capacity = sum(instances * max_aug_per_image for _, instances in eligible)
    if total_capacity < deficit:
        print(
            f"Peringatan: kapasitas augmentasi maksimum ({total_capacity}) lebih kecil dari kebutuhan ({deficit}). "
            "Dataset mungkin tetap tidak seimbang."
        )

    idx = 0
    iterations = 0
    max_iterations = len(eligible) * max_aug_per_image if eligible else 0

    while deficit > 0 and idx < max_iterations and eligible:
        stem, instances = eligible[idx % len(eligible)]
        if plan.get(stem, 0) >= max_aug_per_image:
            idx += 1
            iterations += 1
            continue

        plan[stem] = plan.get(stem, 0) + 1
        deficit -= instances
        idx += 1
        iterations += 1

    if deficit > 0:
        print(f"Peringatan: Masih ada selisih {deficit} instance setelah perencanaan augmentasi.")

    return plan

print("Fungsi utilitas augmentasi dan analisis dataset berhasil didefinisikan!")

In [ ]:
# Definisi pipeline augmentasi
# Transformasi geometri (flip, rotate) ditangani secara manual
# Albumentations hanya untuk transformasi pixel/color yang tidak ubah koordinat
import albumentations as A

transform_color = A.Compose([
    A.RandomBrightnessContrast(
        brightness_limit=0.2, 
        contrast_limit=0.2, 
        p=0.5
    ),
    A.HueSaturationValue(
        hue_shift_limit=10,
        sat_shift_limit=20,
        val_shift_limit=10,
        p=0.5
    ),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.GaussianBlur(blur_limit=(3, 7), p=0.3),
    A.CLAHE(clip_limit=2.0, p=0.3),
], bbox_params=None)

print("Pipeline augmentasi berhasil didefinisikan!")
print("\nTeknik augmentasi yang digunakan:")
print("Transformasi Geometri (dengan transformasi koordinat):")
print("  1. HorizontalFlip (50%)")
print("  2. VerticalFlip (50%)")
print("  3. RandomRotate90 (50%)")
print("\nTransformasi Warna/Pixel:")
print("  4. RandomBrightnessContrast (50%)")
print("  5. HueSaturationValue (50%)")
print("  6. GaussNoise (30%)")
print("  7. GaussianBlur (30%)")
print("  8. CLAHE (30%)")

In [ ]:
# Visualisasi hasil augmentasi
import matplotlib.pyplot as plt
from PIL import Image
import random
import os
from pathlib import Path

def visualize_augmentations(images_path, num_samples=4):
    """Visualisasi gambar original dan hasil augmentasinya"""
    images_dir = Path(images_path)
    
    # Dapatkan gambar original (bukan hasil augmentasi)
    original_images = [f for f in os.listdir(images_path) if '_aug' not in f and f.endswith(('.jpg', '.jpeg', '.png'))]
    
    if len(original_images) == 0:
        print("Tidak ada gambar original ditemukan")
        return
    
    # Pilih beberapa gambar secara random
    sample_images = random.sample(original_images, min(num_samples, len(original_images)))
    
    for orig_name in sample_images:
        # Cari semua augmentasi dari gambar ini
        base_name = Path(orig_name).stem
        aug_images = [f for f in os.listdir(images_path) if f.startswith(base_name + '_aug')]
        
        # Siapkan plot
        num_cols = min(4, len(aug_images) + 1)
        fig, axes = plt.subplots(1, num_cols, figsize=(20, 5))
        
        if num_cols == 1:
            axes = [axes]
        
        # Tampilkan gambar original
        orig_img = Image.open(os.path.join(images_path, orig_name))
        axes[0].imshow(orig_img)
        axes[0].set_title('Original')
        axes[0].axis('off')
        
        # Tampilkan gambar hasil augmentasi
        for idx, aug_name in enumerate(aug_images[:num_cols-1]):
            aug_img = Image.open(os.path.join(images_path, aug_name))
            axes[idx+1].imshow(aug_img)
            axes[idx+1].set_title(f'Augmented {idx+1}')
            axes[idx+1].axis('off')
        
        plt.tight_layout()
        plt.show()
        print(f"Gambar: {orig_name}")
        print("-" * 50)

# NOTE: the actual call to visualize_augmentations is moved later to after augmentation run/verification

In [ ]:
# Konfigurasi path dan parameter balancing
from pathlib import Path

# Gunakan path dataset dari hasil download Roboflow jika tersedia
if 'dataset' in globals():
    DATASET_PATH = Path(dataset.location)
else:
    DATASET_PATH = Path("/content/datasets/Rice-Grain-4")

DATA_CONFIG_PATH = DATASET_PATH / "data.yaml"
data_config = load_data_config(DATA_CONFIG_PATH)
CLASS_NAMES = data_config["names"]
SPLITS = data_config["splits"]

if "train" not in SPLITS:
    raise ValueError("Path train tidak ditemukan pada data.yaml. Pastikan konfigurasi dataset benar.")

TRAIN_IMAGES_PATH = SPLITS["train"]["images"]
TRAIN_LABELS_PATH = SPLITS["train"]["labels"]
VAL_IMAGES_PATH = SPLITS.get("val", {}).get("images")
VAL_LABELS_PATH = SPLITS.get("val", {}).get("labels")
TEST_IMAGES_PATH = SPLITS.get("test", {}).get("images")
TEST_LABELS_PATH = SPLITS.get("test", {}).get("labels")

# Parameter balancing
TARGET_MINORITY_CLASS_NAME = "brown_spot"
DEFAULT_BASE_AUGMENTATIONS = 0  # augmentasi default untuk semua gambar
MAX_AUG_PER_IMAGE = 5           # batas augmentasi tambahan per gambar minoritas

print(f"Dataset path: {DATASET_PATH}")
print(f"Train images path: {TRAIN_IMAGES_PATH}")
print(f"Train labels path: {TRAIN_LABELS_PATH}")
if VAL_IMAGES_PATH:
    print(f"Validation images path: {VAL_IMAGES_PATH}")
if TEST_IMAGES_PATH:
    print(f"Test images path: {TEST_IMAGES_PATH}")
print(f"Jumlah kelas: {len(CLASS_NAMES)} -> {CLASS_NAMES}")

In [ ]:
# Analisis distribusi kelas & rencana balancing
NUM_CLASSES = len(CLASS_NAMES)

train_counts, TRAIN_IMAGE_STATS = collect_class_stats(TRAIN_LABELS_PATH, NUM_CLASSES)
val_counts = Counter({cls: 0 for cls in range(NUM_CLASSES)})
test_counts = Counter({cls: 0 for cls in range(NUM_CLASSES)})

if VAL_LABELS_PATH:
    val_counts, _ = collect_class_stats(VAL_LABELS_PATH, NUM_CLASSES)
if TEST_LABELS_PATH:
    test_counts, _ = collect_class_stats(TEST_LABELS_PATH, NUM_CLASSES)

print_class_distribution("Train (sebelum augmentasi)", train_counts, CLASS_NAMES)
if VAL_LABELS_PATH:
    print_class_distribution("Validation", val_counts, CLASS_NAMES)
if TEST_LABELS_PATH:
    print_class_distribution("Test", test_counts, CLASS_NAMES)

if TARGET_MINORITY_CLASS_NAME in CLASS_NAMES:
    minority_class_id = CLASS_NAMES.index(TARGET_MINORITY_CLASS_NAME)
else:
    minority_class_id = min(train_counts, key=train_counts.get)
majority_class_id = max(train_counts, key=train_counts.get)

balance_deficit = train_counts[majority_class_id] - train_counts[minority_class_id]

print("\nRingkasan balancing:")
print(f"Kelas mayoritas : {CLASS_NAMES[majority_class_id]} ({train_counts[majority_class_id]} instance)")
print(f"Kelas minoritas : {CLASS_NAMES[minority_class_id]} ({train_counts[minority_class_id]} instance)")
print(f"Selisih instance : {balance_deficit}")

AUGMENTATION_PLAN = build_balanced_augmentation_plan(
    TRAIN_IMAGE_STATS,
    target_class_id=minority_class_id,
    deficit=balance_deficit,
    max_aug_per_image=MAX_AUG_PER_IMAGE,
)

expected_new_minority = sum(
    TRAIN_IMAGE_STATS[stem].get(minority_class_id, 0) * count
    for stem, count in AUGMENTATION_PLAN.items()
)

print(f"\nRencana augmentasi untuk kelas {CLASS_NAMES[minority_class_id]}:")
print(f"  - Jumlah gambar unik yang ditambah: {len(AUGMENTATION_PLAN)}")
print(f"  - Total augmentasi baru: {sum(AUGMENTATION_PLAN.values())}")
print(f"  - Perkiraan penambahan instance minoritas: {expected_new_minority}")
if balance_deficit > 0 and not AUGMENTATION_PLAN:
    print("  ! Tidak ada gambar minoritas yang tersedia untuk di-augment. Dataset tetap tidak seimbang.")

BASELINE_TRAIN_COUNTS = train_counts.copy()
MINORITY_CLASS_ID = minority_class_id
MAJORITY_CLASS_ID = majority_class_id

In [ ]:
# Fungsi untuk melakukan augmentasi pada segmentation dataset
def augment_segmentation_dataset(
    images_path,
    labels_path,
    base_num_augmentations=0,
    augmentation_plan=None,
):
    """
    Augmentasi dataset dengan format YOLO segmentation secara class-aware.

    Parameters
    ----------
    images_path : str | Path
        Direktori gambar.
    labels_path : str | Path
        Direktori label (YOLO segmentation).
    base_num_augmentations : int, default 0
        Jumlah augmentasi dasar untuk setiap gambar (dapat 0 jika hanya pakai augmentation_plan).
    augmentation_plan : dict[str, int], optional
        Mapping nama file (tanpa ekstensi) -> jumlah augmentasi tambahan untuk balancing kelas.
    """
    images_dir = Path(images_path)
    labels_dir = Path(labels_path)
    augmentation_plan = augmentation_plan or {}

    image_files = sorted(list(images_dir.glob('*.jpg')) + list(images_dir.glob('*.jpeg')) + list(images_dir.glob('*.png')))

    print(f"Ditemukan {len(image_files)} gambar untuk evaluasi augmentasi")

    augmented_count = 0

    for img_path in tqdm(image_files, desc="Augmentasi gambar"):
        image = cv2.imread(str(img_path))
        if image is None:
            print(f"Gagal membaca gambar: {img_path}")
            continue

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        img_height, img_width = image.shape[:2]

        label_path = labels_dir / f"{img_path.stem}.txt"
        if not label_path.exists():
            print(f"Tidak ada label untuk: {img_path.name}")
            continue

        annotations = read_yolo_annotation(label_path)
        if len(annotations) == 0:
            continue

        planned_augmentations = augmentation_plan.get(img_path.stem, base_num_augmentations)
        if planned_augmentations <= 0:
            continue

        next_aug_index = 0
        while (images_dir / f"{img_path.stem}_aug{next_aug_index}{img_path.suffix}").exists() or \
                (labels_dir / f"{img_path.stem}_aug{next_aug_index}.txt").exists():
            next_aug_index += 1

        for aug_round in range(planned_augmentations):
            augmented_image, transformed_annotations = apply_tracked_transform(
                image.copy(), annotations, img_width, img_height
            )

            color_transformed = transform_color(image=augmented_image)
            augmented_image = color_transformed['image']

            current_index = next_aug_index + aug_round
            aug_img_name = f"{img_path.stem}_aug{current_index}{img_path.suffix}"
            aug_img_path = images_dir / aug_img_name

            augmented_image_bgr = cv2.cvtColor(augmented_image, cv2.COLOR_RGB2BGR)
            cv2.imwrite(str(aug_img_path), augmented_image_bgr)

            aug_label_name = f"{img_path.stem}_aug{current_index}.txt"
            aug_label_path = labels_dir / aug_label_name
            write_yolo_annotation(aug_label_path, transformed_annotations)

            augmented_count += 1

    print(f"\nSelesai! Total {augmented_count} gambar baru telah dibuat")
    print(f"Dataset sekarang memiliki {len(image_files) + augmented_count} gambar training")

    return augmented_count

print("Fungsi augment_segmentation_dataset (class-aware) berhasil didefinisikan!")

In [ ]:
# Jalankan augmentasi
print("Memulai proses augmentasi dengan balancing kelas...")
print("=" * 50)

augmented_count = augment_segmentation_dataset(
    images_path=TRAIN_IMAGES_PATH,
    labels_path=TRAIN_LABELS_PATH,
    base_num_augmentations=DEFAULT_BASE_AUGMENTATIONS,
    augmentation_plan=AUGMENTATION_PLAN,
)

print("=" * 50)
print("Augmentasi selesai!")
print(f"Total gambar baru: {augmented_count}")

In [ ]:
# Verifikasi hasil augmentasi dan balancing
train_counts_after, _ = collect_class_stats(TRAIN_LABELS_PATH, len(CLASS_NAMES))

if VAL_LABELS_PATH:
    val_counts_after, _ = collect_class_stats(VAL_LABELS_PATH, len(CLASS_NAMES))
else:
    val_counts_after = Counter({cls: 0 for cls in range(len(CLASS_NAMES))})

if TEST_LABELS_PATH:
    test_counts_after, _ = collect_class_stats(TEST_LABELS_PATH, len(CLASS_NAMES))
else:
    test_counts_after = Counter({cls: 0 for cls in range(len(CLASS_NAMES))})

print("Statistik Dataset Setelah Augmentasi:")
print("=" * 50)
print(f"Path dataset: {DATASET_PATH}")
print(f"Train images path: {TRAIN_IMAGES_PATH}")
print(f"Train labels path: {TRAIN_LABELS_PATH}")

if 'BASELINE_TRAIN_COUNTS' in globals():
    print("\nPerbandingan distribusi kelas (train):")
    for idx, class_name in enumerate(CLASS_NAMES):
        before = BASELINE_TRAIN_COUNTS.get(idx, 0)
        after = train_counts_after.get(idx, 0)
        delta = after - before
        sign = "+" if delta >= 0 else ""
        print(f"  - {class_name}: sebelum={before}, sesudah={after} ({sign}{delta})")
else:
    print_class_distribution("Train (setelah augmentasi)", train_counts_after, CLASS_NAMES)

if VAL_LABELS_PATH:
    print_class_distribution("Validation", val_counts_after, CLASS_NAMES)
if TEST_LABELS_PATH:
    print_class_distribution("Test", test_counts_after, CLASS_NAMES)

print("\nContoh file hasil augmentasi:")
aug_files = [
    f for f in sorted(TRAIN_IMAGES_PATH.glob('*'))
    if '_aug' in f.stem and f.suffix.lower() in {'.jpg', '.jpeg', '.png'}
]
for i, path in enumerate(aug_files[:5]):
    print(f"  {i+1}. {path.name}")

In [ ]:
# Tampilkan visualisasi hasil augmentasi
visualize_augmentations(TRAIN_IMAGES_PATH, num_samples=2)